In [1]:
import numpy as np
import numba
from numba import jit

In [2]:
print(numba.__version__)

0.51.2


In [3]:
@jit(nopython=True)
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    # assuming square input matrix
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

In [4]:
%%time
x = np.arange(100).reshape(10, 10)
go_fast(x)

CPU times: user 582 ms, sys: 580 ms, total: 1.16 s
Wall time: 467 ms


array([[  9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.],
       [ 19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.],
       [ 29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.],
       [ 39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.],
       [ 49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.],
       [ 59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.],
       [ 69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.],
       [ 79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.],
       [ 89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.],
       [ 99., 100., 101., 102., 103., 104., 105., 106., 107., 108.]])

In [5]:
%%time
go_fast(2*x)

CPU times: user 114 µs, sys: 73 µs, total: 187 µs
Wall time: 662 µs


array([[  9.,  11.,  13.,  15.,  17.,  19.,  21.,  23.,  25.,  27.],
       [ 29.,  31.,  33.,  35.,  37.,  39.,  41.,  43.,  45.,  47.],
       [ 49.,  51.,  53.,  55.,  57.,  59.,  61.,  63.,  65.,  67.],
       [ 69.,  71.,  73.,  75.,  77.,  79.,  81.,  83.,  85.,  87.],
       [ 89.,  91.,  93.,  95.,  97.,  99., 101., 103., 105., 107.],
       [109., 111., 113., 115., 117., 119., 121., 123., 125., 127.],
       [129., 131., 133., 135., 137., 139., 141., 143., 145., 147.],
       [149., 151., 153., 155., 157., 159., 161., 163., 165., 167.],
       [169., 171., 173., 175., 177., 179., 181., 183., 185., 187.],
       [189., 191., 193., 195., 197., 199., 201., 203., 205., 207.]])

In [6]:
%timeit go_fast(x)

740 ns ± 38.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
np.testing.assert_array_equal(go_fast(x), go_fast.py_func(x))

In [8]:
%timeit go_fast.py_func(x)

25 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
def go_numpy(a):
    return a + np.tanh(np.diagonal(a)).sum()

In [10]:
np.testing.assert_array_equal(go_numpy(x), go_fast(x))

In [11]:
%timeit go_numpy(x)

10 µs ± 216 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
import random

@jit(nopython=True)
def spherical_to_cartesian(r, theta, phi):
    '''Convert spherical coordinates (physics convention) to cartesian coordinates'''
    sin_theta = np.sin(theta)
    x = r * sin_theta * np.cos(phi)
    y = r * sin_theta * np.sin(phi)
    z = r * np.cos(theta)
    
    return x, y, z # return a tuple
    
@jit(nopython=True)
def random_directions(n, r):
    '''Return ``n`` 3-vectors in random directions with radius ``r``'''
    out = np.empty(shape=(n,3), dtype=np.float64)
    
    for i in range(n):
        # Pick directions randomly in solid angle
        phi = random.uniform(0, 2*np.pi)
        theta = np.arccos(random.uniform(-1, 1))
        # unpack a tuple
        x, y, z = spherical_to_cartesian(r, theta, phi)
        out[i] = x, y, z
    
    return out

In [13]:
random_directions(10, 1.0)

array([[ 2.83358139e-01,  8.60750546e-01,  4.22867192e-01],
       [-3.25549482e-02,  4.21686897e-01,  9.06156905e-01],
       [ 8.38610387e-01,  1.33190718e-01,  5.28197739e-01],
       [-6.49576271e-02,  5.94645114e-01,  8.01359904e-01],
       [-5.39746049e-01,  2.05837460e-02, -8.41576207e-01],
       [ 7.92680791e-01, -3.85450301e-04,  6.09636790e-01],
       [-9.33625919e-01,  6.26203162e-02,  3.52734091e-01],
       [ 6.35928294e-01, -5.25613502e-01,  5.65089066e-01],
       [-2.13929767e-01,  9.50388488e-01, -2.25822445e-01],
       [ 3.99241260e-01,  8.90770698e-01, -2.17103618e-01]])

In [14]:
from numba import njit, config, __version__
from numba.extending import overload
import numpy as np
assert tuple(int(x) for x in __version__.split('.')[:2]) >= (0, 41)

In [15]:
%%time
%%timeit
if config.PYVERSION > (3, 4): # Only supported in Python >= 3.4
    
    @njit
    def strings_demo(str1, str2, str3):
        # strings, ---^  ---^   ---^
        # as arguments are now supported!
        
        # defining strings in compiled code also works
        def1 = 'numba is '
        
        # as do unicode strings
        def2 = '🐍⚡'
        
        # also string concatenation 
        print(str1 + str2)
        
        # comparison operations
        print(str1 == str2)
        print(str1 < str2)
        print(str1 <= str2)
        print(str1 > str2)
        print(str1 >= str2)
        
        # {starts,ends}with
        print(str1.startswith(str3))
        print(str2.endswith(str3))
        
        # len()
        print(len(str1), len(def2), len(str3))
        
        # str.find()
        print(str2.find(str3))
        
        # in
        print(str3 in str2)
        
        # slicing
        print(str2[1:], str1[:1])
        
        # and finally, strings can also be returned
        return '\nnum' + str1[1::-1] + def1[5:] + def2
    
    
    # run the demo
    print(strings_demo('abc', 'zba', 'a'))

abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
abczba
False
True
True
False
False
True
True
3 2 1
2
True
ba a

numba is 🐍⚡
504 ms ± 9.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU times: user 6.53 s, sys: 24.3 ms, total: 6.56 s
Wall time: 6.55 s


In [16]:
from numba import prange # import parallel range

# decorate a function with `parallel=True` as usual
@njit(parallel=True)
def test(x):
    n = x.shape[0]
    a = np.sin(x)                      # parallel array expression
    b = np.cos(a * a)                  # parallel array expression
    acc = 0                            
    for i in prange(n - 2):            # user defined parallel loop
        for j in prange(n - 1):        # user defined parallel loop
            acc += b[i] + b[j + 1]     # parallel reduction
    return acc

# run the function
test(np.arange(10))

# access the diagnostic output via the new `parallel_diagnostics` method on the dispatcher
test.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function test, <ipython-
input-16-4feb4fd81cc0> (4)  


Parallel loop listing for  Function test, <ipython-input-16-4feb4fd81cc0> (4) 
-----------------------------------------------------------------------|loop #ID
@njit(parallel=True)                                                   | 
def test(x):                                                           | 
    n = x.shape[0]                                                     | 
    a = np.sin(x)                      # parallel array expression-----| #0
    b = np.cos(a * a)                  # parallel array expression-----| #1
    acc = 0                                                            | 
    for i in prange(n - 2):            # user defined parallel loop----| #3
        for j in prange(n - 1):        # user defined parallel loop----| #2
            acc += b[i] + b[j + 1]     # parallel reduction            | 
    return acc                                                         | 

In [17]:
SQRT_2PI = np.sqrt(2 * np.pi)

@jit(nopython=True, parallel=True)
def gaussians(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result / SQRT_2PI / n

In [18]:
means = np.random.uniform(-1, 1, size=1000000)
widths = np.random.uniform(0.1, 0.3, size=1000000)

gaussians(0.4, means, widths)

array([7.10495241e-09, 5.37933081e-08, 1.81712324e-09, ...,
       1.40464309e-11, 3.70438473e-08, 8.54394732e-13])

In [19]:
gaussians_nothread = jit(nopython=True)(gaussians.py_func)

%timeit gaussians_nothread(0.4, means, widths)
%timeit gaussians(0.4, means, widths)

14.4 ms ± 848 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.99 ms ± 472 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit gaussians.py_func(0.4, means, widths) # compare to pure NumPy

17.2 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
@jit(nopython=True, parallel=True)
def kde(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result.mean() / SQRT_2PI

kde_nothread = jit(nopython=True)(kde.py_func)

In [22]:
%timeit kde_nothread(0.4, means, widths)
%timeit kde(0.4, means, widths)

8.8 ms ± 279 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.73 ms ± 768 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
import random

# Serial version
@jit(nopython=True)
def monte_carlo_pi_serial(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

# Parallel version
@jit(nopython=True, parallel=True)
def monte_carlo_pi_parallel(nsamples):
    acc = 0
    # Only change is here
    for i in numba.prange(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [24]:
%time monte_carlo_pi_serial(int(4e8))
%time monte_carlo_pi_parallel(int(4e8))

CPU times: user 3.59 s, sys: 5.6 ms, total: 3.59 s
Wall time: 3.59 s
CPU times: user 6.68 s, sys: 39.3 ms, total: 6.72 s
Wall time: 1.21 s


3.14172985